In [1]:
import logging
import configparser
from sqlalchemy import create_engine, text
import pandas as pd
import datetime as dt

In [2]:
#### IMPORTANTE:
### Configurando caminho para que uma tabela 
### com dados de raios selecionados seja exportada:
output = '/home/andre/Desktop/PCI-INPE/01-relatorio_jan/'

In [4]:
#### IMPORTANTE:
### Ao executar esta célula será pedido o tempo final (sintaxe: "AAAA-MM-DD HH:MM:SS") a
### partir do qual serão "puxados" os dados do banco de dados. 
### Ex: se tempo final = "2024-01-13 00:00:00" e time_interval_minutes = 1440 no arquivo .cfg,
### serão puxadas as localizações dos raios ocorridos entre 12/01/2024 às 00:00:00 e tempo final.


### Configuração do logging para exibir no terminal:
config = configparser.ConfigParser()
config.read('Parametros_SQL.cfg')
logging.basicConfig(level=getattr(logging, config['logging']['log_level']), 
                    format='%(asctime)s - %(levelname)s - %(message)s')

#############################################################
### Funções de Conexão e Carregamento de Dados:
def connect_to_db():
    try:
        engine = create_engine(config['database']['database_url'])
        logging.info("Engine do SQLAlchemy criado com sucesso.")
        return engine
    except Exception as e:
        logging.error(f"Erro ao conectar ao criar a engine: {e}")
        return None   
    
def load_data(engine, table_name, current_time):
    start_time = current_time - dt.timedelta(minutes=int(config['time_settings']['time_interval_minutes']))
    query = f"""
    SELECT latitude, longitude, time, type_data 
    FROM {table_name}
    WHERE time >= '{start_time}' AND time < '{current_time}'
    """
    data = pd.read_sql_query(con=engine.connect(), sql=text(query))
    logging.info(f"Dados carregados da tabela {table_name} para o intervalo {start_time} - {current_time}.")
    return data

#############################################################
### Função para pré-rocessamento dos dados:
def preprocess_data(data, current_time, time_interval_minutes, num_intervals):
    data['time'] = pd.to_datetime(data['time'])
    data = data.dropna()
    data['type_data'] = data['type_data'].astype(int)
    start_time = current_time - dt.timedelta(minutes=time_interval_minutes)
    interval_size = time_interval_minutes // num_intervals
    bins = [start_time + dt.timedelta(minutes=i*interval_size) for i in range(num_intervals + 1)]
    labels = [f'{i*interval_size}-{(i+1)*interval_size}' for i in range(num_intervals)]
    data['time_interval'] = pd.cut(data['time'], bins=bins, labels=labels, right=False)
    
    data['time'] = pd.to_datetime(data['time'])
    data.set_index('time', inplace=True)
    
    logging.info("Dados pré-processados para análise.")
    return data

#############################################################
### Função Principal:
logging.info("Iniciando....")

## Converter o argumento current_time para um objeto datetime
now = input('Escolha o tempo final: ')
simulated_time = dt.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
    
engine = connect_to_db()
if engine is not None:
    logging.info("Conexão com o banco de dados estabelecida com sucesso.")
    
    ## Ler variáveis da configuração:
    region = config['region_settings']['region'].lower()
    time_interval_minutes = int(config['time_settings']['time_interval_minutes'])
    num_intervals = int(config['time_settings']['num_intervals'])
    
    ## Condicional para pegar a tabela certa (sudeste ou sul):
    if region == 'sul':
        table_name = config['region_settings']['table_name_sul']
        extent = list(map(float, config['region_settings']['extent_sul'].split(',')))
    elif region == 'sudeste':
        table_name = config['region_settings']['table_name_sudeste']
        extent = list(map(float, config['region_settings']['extent_sudeste'].split(',')))
    else:
        logging.error("Região não reconhecida. Use 'sul' ou 'sudeste'.")
    logging.info("simulated_time = {0}".format(simulated_time))
    data = load_data(engine, table_name, simulated_time)
    dataframe = preprocess_data(data, simulated_time, time_interval_minutes, num_intervals)
else:
    logging.error("Falha na conexão com o banco de dados.")

2025-02-23 13:51:41,111 - INFO - Iniciando....


Escolha o tempo final: 2024-01-13 00:00:00


2025-02-23 13:51:43,614 - INFO - Engine do SQLAlchemy criado com sucesso.
2025-02-23 13:51:43,614 - INFO - Conexão com o banco de dados estabelecida com sucesso.
2025-02-23 13:51:43,615 - INFO - simulated_time = 2024-01-13 00:00:00
2025-02-23 13:51:49,549 - INFO - Dados carregados da tabela en_brasildat_sample_se para o intervalo 2024-01-12 00:00:00 - 2024-01-13 00:00:00.
2025-02-23 13:51:49,576 - INFO - Dados pré-processados para análise.


In [5]:
### Visualizando um dataframe com os dados "puxados" da tabela SQL:
dataframe

,latitude,longitude,type_data,time_interval
time,,,,
2024-01-12 00:16:56.890641,-20.100300,-52.012400,40,15-20
2024-01-12 00:00:46.316366,-25.263799,-48.303057,1,0-5
2024-01-12 00:00:59.948344,-25.306743,-51.344258,1,0-5
2024-01-12 00:01:01.509948,-23.874586,-45.596715,1,0-5
2024-01-12 00:01:35.017595,-24.804828,-48.948464,1,0-5
...,...,...,...,...
2024-01-12 23:59:57.849225,-21.051954,-51.279332,1,1435-1440
2024-01-12 23:59:58.350264,-21.735211,-51.032495,1,1435-1440
2024-01-12 23:59:58.482074,-21.711463,-51.030607,1,1435-1440


In [6]:
### Exportando e salvando o dataframe em um arquivo pickle:
dataframe.to_pickle(output+'/12_01_24_completo.pickle')